In [0]:
%pip install dbldatagen

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install Faker

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from dbldatagen import DataGenerator , PyfuncText
from faker import Faker
from datetime import timedelta, datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, isnan, col, lit, countDistinct
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType


interval = timedelta(days=1, hours=1)
start = datetime(2020, 10, 1, 0, 0, 0)
end = datetime(2023, 10, 1, 6, 0, 0)

partitions_requested = 20
data_rows = 100

schema = StructType([
    StructField("name", StringType(), False),
    StructField("address", StringType(), True),
    StructField("email", StringType(), True),
    StructField("ip_address", StringType(), True),
    StructField("card_type", StringType(), True),
    StructField("card_num", StringType(), True),
    StructField("Last_Login", TimestampType(), False),

])

ip_address_generator = (lambda context, v : Faker(locale="en_US").ipv4_private())
name_generator = (lambda context, v : Faker(locale="en_US").name())
address_generator = (lambda context, v : Faker(locale="en_US").address())
email_generator = (lambda context, v : Faker(locale="en_US").ascii_company_email())


INFO: Version : VersionInfo(major='0', minor='3', patch='3', release='post', build='2')


In [0]:

fake_data = (DataGenerator(spark,rows=data_rows, partitions=partitions_requested)
            .withSchema(schema)
            .withColumnSpec("name",
                        percentNulls=0.1,
                        text=PyfuncText(name_generator ))
            .withColumnSpec("address",
                        text=PyfuncText(address_generator))
            .withColumnSpec("email",
                        text=PyfuncText(email_generator))
            .withColumnSpec("ip_address",
                        text=PyfuncText(ip_address_generator ))
             .withColumn("int_payment_instrument", "int",  minValue=0, maxValue=9999,  
                baseColumn="name",
                baseColumnType="hash", omit=True)
            .withColumnSpec("card_type", values=['paypal', 'visa', 'mastercard', 'amex'],
                random=True)
            .withColumnSpec("card_num", 
                expr="format_number(int_payment_instrument, '**** ****** *####')",
                baseColumn="int_payment_instrument")
            .withColumnSpec("Last_Login",  percentNulls=0.3, begin=start, end=end, interval=interval, random=True)
            )

In [0]:
fake_data_build = fake_data.build()
display(fake_data_build.limit(10))
            

name,address,email,ip_address,card_type,card_num,Last_Login
Elizabeth Martin,"25543 Lisa Mall North Mary, MO 76784",mleon@austin.net,172.24.169.177,amex,**** ****** *6214,null
Vanessa Oneill,"59646 Martinez Burgs South Jessica, VT 08697",james58@ritter-harrington.biz,10.219.119.183,visa,**** ****** *9648,2021-04-11T17:00:00.000+0000
John Smith,"3336 James Walk Apt. 354 Schneiderstad, MA 81269",ftorres@newman.com,10.208.195.206,visa,**** ****** *1135,2022-11-29T14:00:00.000+0000
Timothy Castillo,"92719 Garcia Freeway South David, MA 85641",james95@hicks-cunningham.com,172.29.38.177,mastercard,**** ****** *95,null
Vanessa Gonzalez,"88936 Cuevas Knoll South Angela, LA 06566",jeffreycollins@kent.info,192.168.163.59,visa,**** ****** *3079,2023-05-02T18:00:00.000+0000
Maria Morris,"45604 Kayla Divide Apt. 718 Port Erichaven, MO 19240",edward60@tucker.com,192.168.206.192,mastercard,**** ****** *5619,null
Mark Hickman,"66827 Watts Mills Suite 517 New Samantha, OK 72247",xjones@mason.info,10.152.22.142,amex,**** ****** *1056,2020-12-29T14:00:00.000+0000
Elizabeth Rodriguez,"577 Ramirez Corner Apt. 950 Harperport, FL 95069",stephenolsen@vasquez.com,172.25.253.241,amex,**** ****** *3574,2022-08-27T20:00:00.000+0000
Caroline Anderson,"5720 Steven Burgs North Jennifer, ID 22515",cynthiamiller@davis.info,10.233.40.18,paypal,**** ****** *2949,2022-08-21T14:00:00.000+0000
Joshua Thompson,"36132 Shane Valleys New Kendraport, NV 47886",charleskim@klein.info,10.122.60.2,mastercard,**** ****** *8511,2023-03-15T20:00:00.000+0000
